# API gRPC Eigen Example using C++

In the following demo, we will be showing the basics of API gRPC protocol by means of a simple library we have created. This library basically contains two elements in its C++ version:

* A **server** which basically implements an API gRPC interface to communicate with an installed library within it, in our case the Eigen library. This API gRPC interface basically exposes certain functionalities such as adding, subtracting and multiplying Eigen::VectorXd and Eigen::MatrixXd in a simple way.
* A **client** in charge of easing the interaction with the server by means of API gRPC interface specific methods. By using the client library, one does not need to know the specifics of the API gRPC interface. 

So, let's get started by importing our client!

**WARNING!!** - This demo is a C++ demo! In order of being capable of running "interpretable" C++ code, some additional libraries were required to be installed using conda. More details can be found in the Dockerfile of this client and in the docs of the project. Do not expect to just copy this demo and run, since it will not work.

In [ ]:
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")

In [ ]:
#pragma cling load("libapi_eigen_example_grpc_client")

In [ ]:
// Let us create a Client object first. This starts by importing the library
#include <apieigen/grpc/GRPCClient.hpp>

This ``GRPCClient`` class is basically the one which will handle the API gRPC interface with the server, together with the connection itself, the formatting of the request and so on. When constructing the class we must provide as inputs the ``host`` of the server and its ``port``, which is basically the endpoint we are accesssing. For this demo we are running, since we deployed both client and server in a docker-compose task, these containers will recognize each other by their docker container names, since they are in the same docker network. Thus, we will provide us arguments:

* Host: cpp-grpc_server_1 (also '0.0.0.0')
* Port: 50000

The server is exposed by **IP 0.0.0.0** and **port 50000** as per defined in the Dockerfile of the server. Thus, the previous inputs should be provided, although they are also the default values. Nonetheless, in the IP field we will provide the DNS for the sake of showing that DNS values are also accepted.

In [ ]:
// Let us instantiate our client
ansys::grpc::client::GRPCClient client{"cpp-grpc_server_1", 50000};

One initial way to see if everything is working properly, is making use of the ``request_greeting`` method available in the client, which performs a simple handshake with the server, whenever we provide a name. The server will receive the request and respond to it. Let's go ahead and do it!

In [ ]:
// Now, let us check if the connection to the server is adequate by requesting a greeting
client.request_greeting("DummyUser");

Now, we will proceed to perform a simple operation such as adding two ``std::vector<double>``. However, let us suppose that we were not able to do it in the client side. So let us ask the server to do it for us.... using gRPC!

In [ ]:
// Once the connection has been checked, let us start declaring some vectors

#include <vector>

std::vector<double> vec1 = {2.0, 3.0, 4.0, 5.0};
std::vector<double> vec2 = {3.0, 7.0, 2.0, 1.0};

Now, we will call the client method ``add_vectors(...)``, and we will explain the typical process of all interface methods (``add_XXXX(...)`` and ``multiply_XXXX(...)``):

* The client performs some sanity checks to confirm that the inputs provided are as expected. This Demo has some limitations such as: only 1D, 2D ``std::vector<>`` containers are allowed; they must be of type ``double``. Direct interaction with the server (i.e. without a client) and using gRPC is out of the scope of this demo. Doing so could be considered as "impossible" since you would have to serialize your message on your own, following the standard defined in the proto files.

* The client serializes the messages with the inputs provided using generator functions. Our end server is characterized for receiving ``streams`` of messages, which basically represent a list of messages. Each of these messages are serialized following the interface proposed by the proto files, thanks to the automatically generated source code by protobuf.
    * For example our ``Vector`` message is characterized for having the following structure:
    
        ```protobuf
        enum DataType{
            INTEGER = 0;
            DOUBLE = 1;
        }
        ...
        message Vector {
            DataType data_type = 1;         // The kind of data inside the vector: INTEGER, DOUBLE values
            int32 vector_size = 2;          // The number of elements inside our vector
            bytes vector_as_chunk = 3;      // The vector itself as a chunk of bytes
        }
        ```

* When the server receives the messages, it deserializes them and interprets each of the previous fields. Then, the desired vectors to be added are passed to the Eigen library  for the resolution of the demanded operation.

* Once the results of the operation are available, the server serializes the result and responds with the adequate message to the client.
    * For example, according to the proto file, our server receives a stream of Vector messages, and returns a single Vector message (which contains the result of the requested operation):
        ```protobuf
        
        // Adds two vectors
        rpc AddVectors(stream Vector) returns (Vector) {}
        ```

* The client then receives the response, deserializes the message and returns the corresponding result to the end-user as a ``std::vector<>``. Thus, the entire process is like a black-box for the end-user, and does not require to understand what is happening behind the scenes, since the end-user is only interested in the end-result.

Let us now call the method!

In [ ]:
// Let us add the vectors!
client.add_vectors(vec1, vec2)

As mentioned before, there are several other methods implemented:

In [ ]:
// Let us compute the dot product of the vectors!
client.multiply_vectors(vec1, vec2)

Let us show as well operations with 2D std::vector containers (i.e. Matrices).

In [ ]:
// Now, let us do the same for matrices!
std::vector<std::vector<double>> mat1 = {{2.0, 3.0}, {4.0, 5.0}};
std::vector<std::vector<double>> mat2 = {{3.0, 7.0}, {2.0, 1.0}};

In [ ]:
// Let us add the matrices!
client.add_matrices(mat1, mat2)

In [ ]:
// Let us multiply the matrices!
client.multiply_matrices(mat1, mat2)